In [1]:
import MetaTrader5 as mt5
import pandas as pd
import plotly.express as px
import numpy as np
from datetime import datetime, timedelta
from config import password, login, server
import time
import pytz
from probability_calculation import weighted_probability


In [2]:
from utils.timeframe import convertTimeFrame

In [3]:
convertTimeFrame('5m')

5

In [2]:
mt5.initialize(login=login, server=server, password=password)

True

In [31]:
# settings
symbol = 'AUDUSD'
timeframe = mt5.TIMEFRAME_M5
start_pos = 0
num_bars = 40000
volume = 1

fsma_period = 7
ssma_period = 28
timezone = pytz.timezone('America/Sao_Paulo')
date_from = datetime(2023, 10, 9, tzinfo=timezone)
date_to = datetime(2023, 9, 10, tzinfo=timezone)
# Obtendo a data e hora atuais
current_day = datetime.now(tz=timezone).replace(microsecond=0)
one_year_before = current_day - timedelta(days=365)
print(date_from)
print(date_to)
print(current_day)
print(one_year_before)

2023-10-09 00:00:00-03:06
2023-09-10 00:00:00-03:06
2023-10-18 00:28:41-03:00
2022-10-18 00:28:41-03:00


In [22]:
# display EURJPY symbol properties
symbol_info=mt5.symbol_info("WDO$")
print(date_from)

2023-10-09 00:00:00-03:06


In [ ]:
df = pd.DataFrame(bars)[['time', 'open', 'close', 'low', 'high']]
df['time'] = pd.to_datetime(df['time'], unit='s')
df['slow_sma'] = df['close'].rolling(ssma_period).mean()
df['fast_sma'] = df['close'].rolling(fsma_period).mean()
df

In [68]:
bars = mt5.copy_rates_range(symbol, timeframe, current_day, one_year_before)
df = pd.DataFrame(bars)
df


""


In [32]:
bars = mt5.copy_rates_from_pos(symbol, timeframe, start_pos, num_bars)
df = pd.DataFrame(bars)[['time', 'open', 'close', 'low', 'high']]
df['time'] = pd.to_datetime(df['time'], unit='s')
df['slow_sma'] = df['close'].rolling(ssma_period).mean()
df['fast_sma'] = df['close'].rolling(fsma_period).mean()

df.dropna(inplace=True)
df

,time,open,close,low,high,slow_sma,fast_sma
27,2023-04-06 10:20:00,0.67011,0.66990,0.66989,0.67035,0.670284,0.670384
28,2023-04-06 10:25:00,0.66990,0.66989,0.66978,0.66996,0.670298,0.670269
29,2023-04-06 10:30:00,0.66991,0.66973,0.66958,0.66992,0.670304,0.670119
30,2023-04-06 10:35:00,0.66975,0.66972,0.66950,0.66978,0.670291,0.670014
31,2023-04-06 10:40:00,0.66970,0.66990,0.66961,0.66996,0.670290,0.669913
...,...,...,...,...,...,...,...
39995,2023-10-18 05:00:00,0.63745,0.63757,0.63744,0.63773,0.636331,0.637430
39996,2023-10-18 05:05:00,0.63755,0.63792,0.63753,0.63798,0.636411,0.637459
39997,2023-10-18 05:10:00,0.63793,0.63767,0.63760,0.63813,0.636498,0.637521
39998,2023-10-18 05:15:00,0.63768,0.63735,0.63728,0.63768,0.636571,0.637570


In [33]:
# Finding Crossovers
df['prev_fast_sma'] = df['fast_sma'].shift(1)


def find_crossovers(slow_sma, fast_sma, prev_fast_sma):
    if slow_sma < fast_sma and prev_fast_sma < slow_sma:
        return 'bull'
    elif slow_sma > fast_sma and prev_fast_sma > slow_sma:
        return 'bear'
    else:
        return None


df['crossover'] = np.vectorize(find_crossovers)(
    df['slow_sma'], df['fast_sma'], df['prev_fast_sma'])

signal = df[df['crossover'] == 'bull'].copy()
signal

c:\Users\sciensa\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning:

invalid value encountered in find_crossovers (vectorized)



,time,open,close,low,high,slow_sma,fast_sma,prev_fast_sma,crossover
107,2023-04-06 17:00:00,0.66658,0.66690,0.66646,0.66706,0.666331,0.666349,0.666213,bull
213,2023-04-07 01:50:00,0.66727,0.66729,0.66725,0.66735,0.667036,0.667083,0.667013,bull
262,2023-04-07 05:55:00,0.66797,0.66807,0.66795,0.66809,0.667984,0.667997,0.667976,bull
263,2023-04-07 06:00:00,0.66807,0.66820,0.66802,0.66827,0.667999,0.668034,0.667997,bull
329,2023-04-07 11:30:00,0.66825,0.66820,0.66817,0.66827,0.667999,0.668050,0.667991,bull
...,...,...,...,...,...,...,...,...,...
39806,2023-10-17 13:15:00,0.63540,0.63569,0.63540,0.63574,0.635047,0.635076,0.634916,bull
39807,2023-10-17 13:20:00,0.63567,0.63556,0.63549,0.63577,0.635081,0.635197,0.635076,bull
39842,2023-10-17 16:15:00,0.63572,0.63608,0.63562,0.63621,0.634864,0.634981,0.634817,bull
39932,2023-10-17 23:45:00,0.63646,0.63660,0.63646,0.63661,0.636375,0.636400,0.636351,bull


In [ ]:
# visualize close price
fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)

fig.show()

In [6]:
# creating backtest and position classes

class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, sl, tp):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'open'

    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume if self.order_type == 'buy' \
            else (self.open_price - self.close_price) * self.volume
        self.status = 'closed'

    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }


class Strategy:
    def __init__(self, df, starting_balance, volume):
        self.starting_balance = starting_balance
        self.volume = volume
        self.positions = []
        self.data = df

    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df

    def add_position(self, position):
        self.positions.append(position)

        return True

    # logic
    def run(self):
        for i, data in self.data.iterrows():

            if data.crossover == 'bear':
                for position in self.positions:
                    if position.status == 'open':
                        position.close_position(data.time, data.close)

            if data.crossover == 'bull':
                self.add_position(
                    Position(data.time, data.close, 'buy', self.volume, 0, 0))

        return self.get_positions_df()

In [34]:
sma_crossover_strategy = Strategy(df, 5000, 100)
result = sma_crossover_strategy.run()

results = [1 if x > 0 else 0 for x in result['profit']][::-1]

results

[0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,


In [35]:
result

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2023-04-06 17:00:00,0.66690,buy,100,0,0,2023-04-06 20:35:00,0.66744,0.054,closed,5000.054
1,2023-04-07 01:50:00,0.66729,buy,100,0,0,2023-04-07 08:20:00,0.66823,0.094,closed,5000.148
2,2023-04-07 05:55:00,0.66807,buy,100,0,0,2023-04-07 08:20:00,0.66823,0.016,closed,5000.164
3,2023-04-07 06:00:00,0.66820,buy,100,0,0,2023-04-07 08:20:00,0.66823,0.003,closed,5000.167
4,2023-04-07 11:30:00,0.66820,buy,100,0,0,2023-04-07 13:05:00,0.66792,-0.028,closed,5000.139
...,...,...,...,...,...,...,...,...,...,...,...
1027,2023-10-17 13:15:00,0.63569,buy,100,0,0,2023-10-17 14:35:00,0.63379,-0.190,closed,4988.704
1028,2023-10-17 13:20:00,0.63556,buy,100,0,0,2023-10-17 14:35:00,0.63379,-0.177,closed,4988.527
1029,2023-10-17 16:15:00,0.63608,buy,100,0,0,2023-10-17 19:10:00,0.63666,0.058,closed,4988.585
1030,2023-10-17 23:45:00,0.63660,buy,100,0,0,2023-10-18 01:25:00,0.63633,-0.027,closed,4988.558


In [36]:
weighted_probability(results) * 100

3.2002064384260356

In [37]:
px.line(result, x='close_datetime', y='pnl')

c:\Users\sciensa\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [13]:
# visualize close price
fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)

for i, row in result[result['status'] == 'closed'].iterrows():

    if row.profit > 0:
        fig.add_shape(type="line",
                      x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
                      line=dict(color="Green", width=3)
                      )

    elif row.profit < 0:
        fig.add_shape(type="line",
                      x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
                      line=dict(color="Red", width=3)
                      )


fig.show()

c:\Users\sciensa\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [ ]:
while True:
    latest_bars = mt5.copy_rates_from_pos(symbol, timeframe, 0, ssma_period)
    df = pd.DataFrame(latest_bars)[['time', 'open', 'close', 'low', 'high']]
    
    df['slow_sma'] = df['close'].rolling(ssma_period).mean()
    df['fast_sma'] = df['close'].rolling(fsma_period).mean()
    df.dropna(inplace=True)

    df['prev_fast_sma'] = df['fast_sma'].shift(1)
    df['crossover'] = np.vectorize(find_crossovers)(df['slow_sma'], df['fast_sma'], df['prev_fast_sma'])

    df

    latest_crossover = df['crossover'].iloc[-1]

    condition_to_buy = latest_crossover == 'bull'

    condition_to_sell = latest_crossover == 'bear'

    if condition_to_buy:
        order = mt5.order_send(
            symbol=symbol,
            action=mt5.ORDER_BUY,
            volume=volume,
            price=mt5.symbol_info_tick(symbol).ask,
            deviation=20,
            type=mt5.ORDER_MARKET
        )

        if order.retcode != mt5.TRADE_RETCODE_DONE:
            print("Erro ao enviar ordem:", mt5.last_error())

    if condition_to_sell:
        position = mt5.positions_get(symbol=symbol)[0]
        order = mt5.order_send(
            symbol=symbol,
            action=mt5.ORDER_SELL,
            volume=volume,
            price=mt5.symbol_info_tick(symbol).bid,
            deviation=20,
            position=position.ticket,
            type=mt5.ORDER_MARKET
        )

        if order.retcode != mt5.TRADE_RETCODE_DONE:
            print("Erro ao fechar posição:", mt5.last_error())

    time.sleep(60) 

In [109]:
lot = 0.1
point = mt5.symbol_info(symbol).point
price = mt5.symbol_info_tick(symbol).ask
symbol_info = mt5.symbol_info(symbol)
deviation = 20
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": volume,
    "type": mt5.ORDER_TYPE_BUY,
    "price": price,
    "sl": price - 100 * point,
    "tp": price + 100 * point,
    "deviation": deviation,
    "magic": 234000,
    "comment": "python script open",
    "type_time": mt5.ORDER_TIME_GTC,
}
 

order = mt5.order_send(request)

print(order, 'order')

if order.retcode != mt5.TRADE_RETCODE_DONE:
    print(f"Erro ao enviar ordem. Código de retorno: {order.retcode}, Resultado: {order.comment}, Mensagem de Erro: {mt5.last_error()}")


OrderSendResult(retcode=10009, deal=319872549, order=379441897, volume=0.1, price=149.105, bid=149.099, ask=149.105, comment='Request executed', request_id=1890370004, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='USDJPY', volume=0.1, price=149.105, stoplimit=0.0, sl=149.005, tp=149.20499999999998, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0)) order


In [110]:
mt5.orders_get()

()